In [196]:
import tarfile
import numpy as np

import keras
import keras.utils
from keras.utils import np_utils

In [197]:
with open("./data/fer2013.csv") as f:
    content = f.readlines()

    lines = np.array(content)

    num_of_instances = lines.size
    print("number of instances: ",num_of_instances)


number of instances:  35341


In [199]:
x_train, y_train, x_test, y_test = [], [], [], []
num_classes=6

for i in range(1,num_of_instances):
    emotion, img, usage = lines[i].split(",")
    val = img.split(" ")
    pixels = np.array(val, 'float32')

    emotion = keras.utils.np_utils.to_categorical(emotion,num_classes)
    if 'Training' in usage:
        y_train.append(emotion)
        x_train.append(pixels)

    elif 'PublicTest' in usage:
        y_test.append(emotion)
        x_test.append(pixels)

In [200]:
np.shape(x_train), np.shape(y_train), np.shape(x_test), np.shape(y_test)

((28273, 2304), (28273, 6), (3533, 2304), (3533, 6))

In [201]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D,AveragePooling2D

In [202]:

model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))

model.summary()


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 44, 44, 64)        1664      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 18, 18, 64)        36928     
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
average_pooling2d_28 (Averag (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 5, 5, 128)         73856     
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 3, 3, 128)       

In [169]:
from keras.preprocessing.image import ImageDataGenerator

In [170]:
batch_size=16
epochs=10

In [182]:
gen = ImageDataGenerator(rescale=1./255)
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

model.compile(loss='categorical_crossentropy'
            , optimizer=keras.optimizers.Adam()
            , metrics=['accuracy'])

model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs)

ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (2304,), y.shape = (28709, 7)